# Evaluate RAG

In [1]:
import os
import pandas as pd
import sys
import random
sys.path.append("../")
from src.config import Configuration
from src.robot import RAGRobot
from src.service.provider import ProviderService

config = Configuration()
provider = ProviderService()
robot = RAGRobot(provider=provider)

test_folder = "training_program"
rag = robot.training_prop_rag.chain
# rag
config.enable_tracing(project="EVALUATE")

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded index doc
loaded index doc
loaded index doc


In [2]:
path = f"../data/{test_folder}/test"
files = os.listdir(path)
all_files = [f"{path}/{f}" for f in files]
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

df = df[['question', 'answer', 'wrong', 'doc_id']]
df['category'] = "training"
df.shape
# df.head()

(22, 5)

In [3]:
records = []
for i, row in df.iterrows():
    prompt = ""
    labels = ["A", "B", "C", "D"]
    record = {**row.to_dict()}
    question = row['question']
    try:
        choices = row['wrong'].split("\n- ")[1:]
        resp = rag.invoke({"question": question})
    except:
        print("error")
        continue
    # update
    record['pred_doc_id'] = resp.document.id
    record['RAG_answer'] = resp.answer
    rand_idx = random.randint(0, len(labels) - 1)

    record['correct_label'] = labels.pop(rand_idx)
    prompt += f"{record['correct_label']}. {record['answer']}\n"
    while len(labels) > 0:
        rand_idx = random.randint(0, len(labels) - 1)
        prompt += f"{labels.pop(rand_idx)}. {choices.pop()}\n"
    record['multi_prompt'] = prompt
    records.append(record)

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


error
error
error


In [5]:
row['wrong']

nan

In [4]:
ndf = pd.DataFrame(records)
ndf.to_csv(f"../experiments/results/eval_{test_folder}_rag.csv")

In [6]:
ndf.head()

,question,answer,wrong,doc_id,category,pred_doc_id,RAG_answer,correct_label,multi_prompt
0,Trường Đại học Tôn Đức Thắng được thành lập th...,Quyết định 787/TTg-QĐ,- Quyết định 687/TTg-QĐ\n- Quyết định 887/TTg-...,d2324ca0-a1bd-4b91-917b-58f233c5fae6,uni,d2324ca0-a1bd-4b91-917b-58f233c5fae6,Quyết định 787/TTg-QĐ,D,D. Quyết định 787/TTg-QĐ\nB. Quyết định 787/QĐ...
1,Trường Đại học Tôn Đức Thắng hướng đến mục tiê...,Một đại học định hướng nghiên cứu có chất lượn...,- Một trường đại học đào tạo nghề có chất lượn...,d2324ca0-a1bd-4b91-917b-58f233c5fae6,uni,38f5d21a-4fdb-4ceb-81cd-4bc968a10ddb,Trường đại học nghiên cứu,A,A. Một đại học định hướng nghiên cứu có chất l...
2,Tòa nhà ngoại ngữ có bao nhiêu tầng?,6 tầng,- 4 tầng\n- 8 tầng\n- 10 tầng\n- 12 tầng,ba62a0d0-c81e-4fc7-b572-db313e678246,uni,ba62a0d0-c81e-4fc7-b572-db313e678246,6,D,D. 6 tầng\nA. 12 tầng\nB. 10 tầng\nC. 8 tầng\n
3,Hồ bơi có diện tích mặt nước là bao nhiêu?,"13,5m x 25m",- 10m x 20m\n- 15m x 30m\n- 12m x 24m\n- 14m x...,ba62a0d0-c81e-4fc7-b572-db313e678246,uni,ba62a0d0-c81e-4fc7-b572-db313e678246,"13,5m x 25m",C,"C. 13,5m x 25m\nA. 14m x 26m\nB. 12m x 24m\nD...."
4,Trường Đại học Tôn Đức Thắng xếp thứ mấy thế g...,601-800,- 1-100\n- 101-200\n- 201-400\n- 401-600,f8318b1a-44a7-4ead-aad8-069ae609f0f9,uni,f8318b1a-44a7-4ead-aad8-069ae609f0f9,601-800,C,C. 601-800\nD. 401-600\nA. 201-400\nB. 101-200\n
